# Question 1

Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [319]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

In [95]:
def answer_one():
    import pandas as pd
    
    # Load the Energy data from an excel file
    x = pd.ExcelFile("datasets/Energy Indicators.xls")
    # Load to a dataframe 
    Energy = x.parse(skiprows=17, # Skiping the header information 
                     skipfooter =(38)) # Skipping the 38 latest rows
    
    # Keep only the columns we want
    Energy = Energy[["Unnamed: 2", "Petajoules", "Gigajoules", "%"]]
    
    # Change the columns labels
    Energy.columns = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]
    
    # Convert missing data to np.NaN
    missing_data_columns = ["Energy Supply", "Energy Supply per Capita"]
    Energy[missing_data_columns] = Energy[missing_data_columns].replace("...", np.NaN, regex=False)
    
    # Convert Energy Supply to gigajoules (1 Petajoule = 1,000,000 Gigajoule)
    Energy["Energy Supply"] = Energy["Energy Supply"] * 1000000
    
    # Remove parenthesis and numbers content
    Energy["Country"].replace(" \(.*\)","",regex=True, inplace=True)
    Energy["Country"].replace("([0-9]+)$", "", regex=True, inplace=True)
    
    # Rename a list of countries:
    old_names = ["Republic of Korea",
                 "United States of America",
                 "United Kingdom of Great Britain and Northern Ireland",
                 "China, Hong Kong Special Administrative Region"]
    
    new_names = ["South Korea", 
                 "United States", 
                 "United Kingdom", 
                 "Hong Kong"]
    
    for n in range(len(old_names)):
        Energy["Country"].replace(old_names[n], new_names[n], inplace=True)
    
    # Strip countries names:
    Energy["Country"] = [country.strip() for country in Energy["Country"]]
    
    country_list = ['Australia', 'Bolivia', 'China', 'Hong Kong', 'China, Macao Special Administrative Region', 'Denmark', 'Falkland Islands', 'France', 'Greenland', 'Indonesia', 'Iran', 'Italy', 'Japan', 'Kuwait', 'Micronesia', 'Netherlands', 'Portugal', 'South Korea', 'Saudi Arabia', 'Serbia', 'Sint Maarten', 'Spain', 'Switzerland', 'Ukraine', 'United Kingdom', 'United States', 'Venezuela']

    for i in range(len(country_list)):
        if country_list[i] not in Energy.index.values.tolist():
            print(country_list[i])
    #If Country is not in the index, replace index with ['Country] in the above line.
    
    
    # Load GDP data
    GDP = pd.read_csv("datasets/world_bank.csv", skiprows=4)
    
    # Rename Country Name column
    GDP.rename(columns={"Country Name": "Country"}, inplace=True)
    
    # Rename a list of countries:
    old_names = ["Korea, Rep.", 
                 "Iran, Islamic Rep.",
                 "Hong Kong SAR, China"]
    new_names = ["South Korea", 
                 "Iran",
                 "Hong Kong"]
    
    for n in range(len(old_names)): 
        GDP["Country"].replace(old_names[n], new_names[n], inplace=True)
        
    # Load the ScimEn data from an excel file
    x = pd.ExcelFile("datasets/scimagojr-3.xlsx")
    
    # Load to a dataframe
    ScimEn = x.parse()
    
    # Keep only 2006-2015 columns in GDP
    columns_to_keep = ["Country", "2006", "2007", "2008", "2009", "2010", "2011", 
                       "2012", "2013", "2014", "2015"]
    GDP = GDP[columns_to_keep]
    
    # Keep all columns but only the top 15 rank countries in ScimEn
    ScimEn_Top_15 = ScimEn[:15]
    
    # Merge ScimEn and Energy intersection 
    intersection_1 = pd.merge(ScimEn_Top_15, # left dataframe
                              Energy, # right dataframe
                              how="inner", # Intersection of both
                              left_on="Country", # left index to intersect
                              right_on="Country") # right index to intersect
    
    second_merge = pd.merge(intersection_1, # Left dataframe
                            GDP, # Right dataframe
                            how="inner", # Intersection
                            left_on="Country", # Left index to intersect
                            right_on="Country") # Right index to intersect
    # Change index
    second_merge.set_index("Country", inplace=True)
    
    # Sort by Rank
    second_merge.sort_values(by=["Rank"])
    
    return second_merge
    
    raise NotImplementedError()

In [119]:
import pandas as pd

# Load the Energy data from an excel file
x = pd.ExcelFile("datasets/Energy Indicators.xls")
# Load to a dataframe 
Energy = x.parse(skiprows=17, # Skiping the header information 
                 skipfooter =(38)) # Skipping the 38 latest rows

# Keep only the columns we want
Energy = Energy[["Unnamed: 2", "Petajoules", "Gigajoules", "%"]]

# Change the columns labels
Energy.columns = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]

# Convert missing data to np.NaN
missing_data_columns = ["Energy Supply", "Energy Supply per Capita"]
Energy[missing_data_columns] = Energy[missing_data_columns].replace("...", np.NaN, regex=False)

# Convert Energy Supply to gigajoules (1 Petajoule = 1,000,000 Gigajoule)
Energy["Energy Supply"] = Energy["Energy Supply"] * 1000000

# Remove parenthesis and numbers content
Energy["Country"].replace(" \(.*\)","",regex=True, inplace=True)
Energy["Country"].replace("([0-9]+)$", "", regex=True, inplace=True)

# Rename a list of countries:
old_names = ["Republic of Korea",
             "United States of America",
             "United Kingdom of Great Britain and Northern Ireland",
             "China, Hong Kong Special Administrative Region"]

new_names = ["South Korea", 
             "United States", 
             "United Kingdom", 
             "Hong Kong"]

for n in range(len(old_names)):
    Energy["Country"].replace(old_names[n], new_names[n], inplace=True)

# Strip countries names:
Energy["Country"] = [country.strip() for country in Energy["Country"]]


# Change index to Country
Energy.setindex([""])

country_list = ['Australia', 'Bolivia', 'China', 'Hong Kong', 'China, Macao Special Administrative Region', 'Denmark', 'Falkland Islands', 'France', 'Greenland', 'Indonesia', 'Iran', 'Italy', 'Japan', 'Kuwait', 'Micronesia', 'Netherlands', 'Portugal', 'South Korea', 'Saudi Arabia', 'Serbia', 'Sint Maarten', 'Spain', 'Switzerland', 'Ukraine', 'United Kingdom', 'United States', 'Venezuela']

for i in range(len(country_list)):
    if country_list[i] not in Energy.index.values.tolist():
        print(country_list[i])
#If Country is not in the index, replace index with ['Country] in the above line.




"\ncountry_list = ['Australia', 'Bolivia', 'China', 'Hong Kong', 'China, Macao Special Administrative Region', 'Denmark', 'Falkland Islands', 'France', 'Greenland', 'Indonesia', 'Iran', 'Italy', 'Japan', 'Kuwait', 'Micronesia', 'Netherlands', 'Portugal', 'South Korea', 'Saudi Arabia', 'Serbia', 'Sint Maarten', 'Spain', 'Switzerland', 'Ukraine', 'United Kingdom', 'United States', 'Venezuela']\n\nfor i in range(len(country_list)):\n    if country_list[i] not in Energy.index.values.tolist():\n        print(country_list[i])\n#If Country is not in the index, replace index with ['Country] in the above line.\n\n"

In [121]:
print(Energy.index.values.tolist())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [120]:
energy = Energy["Country"]
energy[energy.eq("Spain")]

190    Spain
Name: Country, dtype: object

In [320]:
import pandas as pd

# Load the data from an excel file
x = pd.ExcelFile("DataSets/Energy Indicators.xls")
# Load to a dataframe 
Energy = x.parse(skiprows=17, # Skiping the header information
                 skipfooter =(38)) # Skipping the 38 latest rows
Energy.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Petajoules,Gigajoules,%
0,NaN,Afghanistan,Afghanistan,321,10,78.669280
1,NaN,Albania,Albania,102,35,100.000000
2,NaN,Algeria,Algeria,1959,51,0.551010
3,NaN,American Samoa,American Samoa,...,...,0.641026
4,NaN,Andorra,Andorra,9,121,88.695650


In [321]:
# Look the columns names
Energy.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Petajoules', 'Gigajoules',
       '%'],
      dtype='object')

In [322]:
len(Energy)

227

In [323]:
# Keep only the columns we want
Energy = Energy[["Unnamed: 2", "Petajoules", "Gigajoules", "%"]]

# Change the columns labels
Energy.columns = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]
Energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,321,10,78.669280
1,Albania,102,35,100.000000
2,Algeria,1959,51,0.551010
3,American Samoa,...,...,0.641026
4,Andorra,9,121,88.695650


In [324]:
# Looking for missing data ("...") 
Energy[Energy['Energy Supply'].str.contains("...", na=False)]

,Country,Energy Supply,Energy Supply per Capita,% Renewable
3,American Samoa,...,...,0.641026
86,Guam,...,...,0.000000
150,Northern Mariana Islands,...,...,0.000000
210,Tuvalu,...,...,0.000000
217,United States Virgin Islands,...,...,0.000000


In [325]:
# Convert missing data to np.NaN
missing_data_columns = ["Energy Supply", "Energy Supply per Capita"]
Energy[missing_data_columns] = Energy[missing_data_columns].replace("...", np.NaN, regex=False)
Energy.loc[[3,86,150,210,217]]

,Country,Energy Supply,Energy Supply per Capita,% Renewable
3,American Samoa,NaN,NaN,0.641026
86,Guam,NaN,NaN,0.000000
150,Northern Mariana Islands,NaN,NaN,0.000000
210,Tuvalu,NaN,NaN,0.000000
217,United States Virgin Islands,NaN,NaN,0.000000


In [326]:
# Convert Energy Supply to gigajoules (1 Petajoule = 1,000,000 Gigajoule)
Energy["Energy Supply"] = Energy["Energy Supply"] * 1000000
Energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,3.210000e+08,10.0,78.669280
1,Albania,1.020000e+08,35.0,100.000000
2,Algeria,1.959000e+09,51.0,0.551010
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9.000000e+06,121.0,88.695650


In [327]:
# Remove parenthesis and numbers content
Energy["Country"].replace(" \(.*\)","",regex=True, inplace=True)
Energy["Country"].replace("([0-9]+)$", "", regex=True, inplace=True)

In [328]:
for x in Energy["Country"]:
    print(x)

Afghanistan
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia
Bonaire, Sint Eustatius and Saba
Bosnia and Herzegovina
Botswana
Brazil
British Virgin Islands
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Cayman Islands
Central African Republic
Chad
Chile
China
China, Hong Kong Special Administrative Region
China, Macao Special Administrative Region
Colombia
Comoros
Congo
Cook Islands
Costa Rica
Côte d'Ivoire
Croatia
Cuba
Curaçao
Cyprus
Czech Republic
Democratic People's Republic of Korea
Democratic Republic of the Congo
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Faeroe Islands
Falkland Islands
Fiji
Finland
France
French Guiana
French Polynesia
Gabon
Gambia
Georgia
Germany
Ghana
Gibraltar
Greece
Greenland
Grenada
Guad

In [329]:
# Rename a list of countries:
# "Republic of Korea": "South Korea",
# "United States of America": "United States",
# "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
# "China, Hong Kong Special Administrative Region": "Hong Kong"

old_names = ["Republic of Korea", 
             "United States of America", 
             "United Kingdom of Great Britain and Northern Ireland", 
             "China, Hong Kong Special Administrative Region"]

new_names = ["South Korea", 
             "United States", 
             "United Kingdom", 
             "Hong Kong"]
for n in range(len(old_names)):
    Energy["Country"].replace(old_names[n], new_names[n], inplace=True)

In [331]:
country_list = ['Australia', 'Bolivia', 'China', 'Hong Kong', 'China, Macao Special Administrative Region', 'Denmark', 'Falkland Islands', 'France', 'Greenland', 'Indonesia', 'Iran', 'Italy', 'Japan', 'Kuwait', 'Micronesia', 'Netherlands', 'Portugal', 'South Korea', 'Saudi Arabia', 'Serbia', 'Sint Maarten', 'Spain', 'Switzerland', 'Ukraine', 'United Kingdom', 'United States', 'Venezuela']

for i in range(len(country_list)):
    if country_list[i] not in Energy.index.values.tolist():
        print(country_list[i])
#If Country is not in the index, replace index with ['Country] in the above line.

Australia
Bolivia
China
Hong Kong
China, Macao Special Administrative Region
Denmark
Falkland Islands
France
Greenland
Indonesia
Iran
Italy
Japan
Kuwait
Micronesia
Netherlands
Portugal
South Korea
Saudi Arabia
Serbia
Sint Maarten
Spain
Switzerland
Ukraine
United Kingdom
United States
Venezuela


In [288]:
# Load GDP data
GDP = pd.read_csv("DataSets/world_bank.csv", skiprows=4)
GDP.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.534637e+09,2.727850e+09,2.790849e+09,2.962905e+09,2.983637e+09,3.092430e+09,3.202189e+09,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,1.929193e+10,1.970186e+10,2.147035e+10,2.570500e+10,2.350165e+10,2.678117e+10,...,9.131974e+11,9.276555e+11,9.563187e+11,8.930997e+11,8.547519e+11,9.622690e+11,9.840320e+11,9.778092e+11,8.984741e+11,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,1.980707e+10,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.040732e+10,1.113130e+10,1.194684e+10,1.268022e+10,1.384262e+10,1.486682e+10,...,7.169352e+11,8.078189e+11,8.469431e+11,7.574921e+11,6.874847e+11,6.809891e+11,7.381313e+11,7.920789e+11,7.865850e+11,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.013532e+11,8.941719e+10,6.230691e+10,NaN


In [289]:
GDP.rename(columns={"Country Name": "Country"}, inplace=True)

In [290]:
for name in GDP["Country"]:
    print(name)

Aruba
Africa Eastern and Southern
Afghanistan
Africa Western and Central
Angola
Albania
Andorra
Arab World
United Arab Emirates
Argentina
Armenia
American Samoa
Antigua and Barbuda
Australia
Austria
Azerbaijan
Burundi
Belgium
Benin
Burkina Faso
Bangladesh
Bulgaria
Bahrain
Bahamas, The
Bosnia and Herzegovina
Belarus
Belize
Bermuda
Bolivia
Brazil
Barbados
Brunei Darussalam
Bhutan
Botswana
Central African Republic
Canada
Central Europe and the Baltics
Switzerland
Channel Islands
Chile
China
Cote d'Ivoire
Cameroon
Congo, Dem. Rep.
Congo, Rep.
Colombia
Comoros
Cabo Verde
Costa Rica
Caribbean small states
Cuba
Curacao
Cayman Islands
Cyprus
Czech Republic
Germany
Djibouti
Dominica
Denmark
Dominican Republic
Algeria
East Asia & Pacific (excluding high income)
Early-demographic dividend
East Asia & Pacific
Europe & Central Asia (excluding high income)
Europe & Central Asia
Ecuador
Egypt, Arab Rep.
Euro area
Eritrea
Spain
Estonia
Ethiopia
European Union
Fragile and conflict affected situations
F

In [291]:
# Rename a list of countries:
# "Korea, Rep.": "South Korea", 
# "Iran, Islamic Rep.": "Iran",
# "Hong Kong SAR, China": "Hong Kong"

old_names = ["Korea, Rep.", 
             "Iran, Islamic Rep.", 
             "Hong Kong SAR, China"]

new_names = ["South Korea", 
             "Iran", 
             "Hong Kong"]

for n in range(len(old_names)):
    GDP["Country"].replace(old_names[n], new_names[n], inplace=True)


In [292]:
for name in GDP["Country"]:
    print(name)

Aruba
Africa Eastern and Southern
Afghanistan
Africa Western and Central
Angola
Albania
Andorra
Arab World
United Arab Emirates
Argentina
Armenia
American Samoa
Antigua and Barbuda
Australia
Austria
Azerbaijan
Burundi
Belgium
Benin
Burkina Faso
Bangladesh
Bulgaria
Bahrain
Bahamas, The
Bosnia and Herzegovina
Belarus
Belize
Bermuda
Bolivia
Brazil
Barbados
Brunei Darussalam
Bhutan
Botswana
Central African Republic
Canada
Central Europe and the Baltics
Switzerland
Channel Islands
Chile
China
Cote d'Ivoire
Cameroon
Congo, Dem. Rep.
Congo, Rep.
Colombia
Comoros
Cabo Verde
Costa Rica
Caribbean small states
Cuba
Curacao
Cayman Islands
Cyprus
Czech Republic
Germany
Djibouti
Dominica
Denmark
Dominican Republic
Algeria
East Asia & Pacific (excluding high income)
Early-demographic dividend
East Asia & Pacific
Europe & Central Asia (excluding high income)
Europe & Central Asia
Ecuador
Egypt, Arab Rep.
Euro area
Eritrea
Spain
Estonia
Ethiopia
European Union
Fragile and conflict affected situations
F

In [316]:
len(GDP)

266

In [293]:
# Load the ScimEn data from an excel file
x = pd.ExcelFile("DataSets/scimagojr-3.xlsx")

# Load to a dataframe 
ScimEn = x.parse()
ScimEn.head()

,Rank,Country,Region,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,Asiatic Region,273437,272374,2336764,1615239,8.55,245
1,2,United States,Northern America,175891,172431,2230544,724472,12.68,363
2,3,India,Asiatic Region,55082,53775,463165,162944,8.41,181
3,4,Japan,Asiatic Region,50523,50065,488062,119930,9.66,193
4,5,United Kingdom,Western Europe,43389,42284,615670,111290,14.19,226


In [317]:
len(ScimEn)

208

# 5

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

In [294]:
GDP.head()

,Country,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.534637e+09,2.727850e+09,2.790849e+09,2.962905e+09,2.983637e+09,3.092430e+09,3.202189e+09,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,1.929193e+10,1.970186e+10,2.147035e+10,2.570500e+10,2.350165e+10,2.678117e+10,...,9.131974e+11,9.276555e+11,9.563187e+11,8.930997e+11,8.547519e+11,9.622690e+11,9.840320e+11,9.778092e+11,8.984741e+11,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,1.980707e+10,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.040732e+10,1.113130e+10,1.194684e+10,1.268022e+10,1.384262e+10,1.486682e+10,...,7.169352e+11,8.078189e+11,8.469431e+11,7.574921e+11,6.874847e+11,6.809891e+11,7.381313e+11,7.920789e+11,7.865850e+11,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.013532e+11,8.941719e+10,6.230691e+10,NaN


In [295]:
# Only 2006-2015 in GDP
columns_to_keep = ["Country", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015"]
GDP = GDP[columns_to_keep]

In [296]:
GDP.head()

,Country,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,2.424581e+09,2.615084e+09,2.745251e+09,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.727850e+09,2.790849e+09,2.962905e+09
1,Africa Eastern and Southern,5.321390e+11,6.115618e+11,6.662911e+11,6.692327e+11,8.036886e+11,8.962561e+11,9.131974e+11,9.276555e+11,9.563187e+11,8.930997e+11
2,Afghanistan,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10
3,Africa Western and Central,3.843363e+11,4.518661e+11,5.530312e+11,4.925458e+11,5.802173e+11,6.584282e+11,7.169352e+11,8.078189e+11,8.469431e+11,7.574921e+11
4,Angola,5.238101e+10,6.526645e+10,8.853861e+10,7.030716e+10,8.379950e+10,1.117897e+11,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11


In [297]:
ScimEn.head()

,Rank,Country,Region,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,Asiatic Region,273437,272374,2336764,1615239,8.55,245
1,2,United States,Northern America,175891,172431,2230544,724472,12.68,363
2,3,India,Asiatic Region,55082,53775,463165,162944,8.41,181
3,4,Japan,Asiatic Region,50523,50065,488062,119930,9.66,193
4,5,United Kingdom,Western Europe,43389,42284,615670,111290,14.19,226


In [298]:
# Keep all columns except Region in ScimEn but only the top 15 rank countries
ScimEn.drop("Region", axis=1, inplace=True)
ScimEn_Top_15 = ScimEn[:15]

In [299]:
ScimEn.head(20)

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,273437,272374,2336764,1615239,8.55,245
1,2,United States,175891,172431,2230544,724472,12.68,363
2,3,India,55082,53775,463165,162944,8.41,181
3,4,Japan,50523,50065,488062,119930,9.66,193
4,5,United Kingdom,43389,42284,615670,111290,14.19,226
5,6,Germany,38739,38013,433148,95145,11.18,196
6,7,Russian Federation,36735,36560,115938,54993,3.16,90
7,8,Canada,33472,32863,568080,100953,16.97,227
8,9,Italy,27983,26940,352993,87828,12.61,166
9,10,South Korea,27655,27445,328488,61531,11.88,155


In [300]:
first_merge = pd.merge(ScimEn_Top_15, # left dataframe
                       Energy, # right dataframe
                       how="inner", # Intersection of both
                       left_on="Country", # left index to intersect
                       right_on="Country") # right index to intersect
first_merge.head(15)

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable
0,1,China,273437,272374,2336764,1615239,8.55,245,1.271910e+11,93.0,19.754910
1,2,United States,175891,172431,2230544,724472,12.68,363,9.083800e+10,286.0,11.570980
2,3,India,55082,53775,463165,162944,8.41,181,3.319500e+10,26.0,14.969080
3,4,Japan,50523,50065,488062,119930,9.66,193,1.898400e+10,149.0,10.232820
4,5,United Kingdom,43389,42284,615670,111290,14.19,226,7.920000e+09,124.0,10.600470
5,6,Germany,38739,38013,433148,95145,11.18,196,1.326100e+10,165.0,17.901530
6,7,Russian Federation,36735,36560,115938,54993,3.16,90,3.070900e+10,214.0,17.288680
7,8,Canada,33472,32863,568080,100953,16.97,227,1.043100e+10,296.0,61.945430
8,9,Italy,27983,26940,352993,87828,12.61,166,6.530000e+09,109.0,33.667230
9,10,South Korea,27655,27445,328488,61531,11.88,155,1.100700e+10,221.0,2.279353


In [301]:
second_merge = pd.merge(first_merge, # Left dataframe
                       GDP, # Right dataframe
                       how="inner", # Intersection
                       left_on="Country", # Left index to intersect
                       right_on="Country") # Right index to intersect

# Change index
second_merge.set_index("Country", inplace=True)

# Sort by Rank
second_merge.sort_values(by=["Rank"])

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,273437,272374,2336764,1615239,8.55,245,1.271910e+11,93.0,19.754910,2.752132e+12,3.550343e+12,4.594307e+12,5.101703e+12,6.087164e+12,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13
United States,2,175891,172431,2230544,724472,12.68,363,9.083800e+10,286.0,11.570980,1.381461e+13,1.445186e+13,1.471284e+13,1.444893e+13,1.499205e+13,1.554258e+13,1.619701e+13,1.678485e+13,1.752716e+13,1.823830e+13
India,3,55082,53775,463165,162944,8.41,181,3.319500e+10,26.0,14.969080,9.402599e+11,1.216735e+12,1.198896e+12,1.341887e+12,1.675615e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12
Japan,4,50523,50065,488062,119930,9.66,193,1.898400e+10,149.0,10.232820,4.530377e+12,4.515265e+12,5.037908e+12,5.231383e+12,5.700098e+12,6.157460e+12,6.203213e+12,5.155717e+12,4.850414e+12,4.389476e+12
United Kingdom,5,43389,42284,615670,111290,14.19,226,7.920000e+09,124.0,10.600470,2.706015e+12,3.093407e+12,2.921432e+12,2.412390e+12,2.481580e+12,2.659882e+12,2.704017e+12,2.783251e+12,3.065521e+12,2.932785e+12
Germany,6,38739,38013,433148,95145,11.18,196,1.326100e+10,165.0,17.901530,2.992197e+12,3.421229e+12,3.730028e+12,3.397791e+12,3.396354e+12,3.744409e+12,3.527345e+12,3.732743e+12,3.883920e+12,3.356236e+12
Russian Federation,7,36735,36560,115938,54993,3.16,90,3.070900e+10,214.0,17.288680,9.899305e+11,1.299706e+12,1.660846e+12,1.222644e+12,1.524917e+12,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12
Canada,8,33472,32863,568080,100953,16.97,227,1.043100e+10,296.0,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,27983,26940,352993,87828,12.61,166,6.530000e+09,109.0,33.667230,1.947920e+12,2.210293e+12,2.398857e+12,2.191242e+12,2.134018e+12,2.291991e+12,2.087077e+12,2.141315e+12,2.159134e+12,1.835899e+12


In [44]:
def answer_one():
    import pandas as pd
    
    # Load the Energy data from an excel file
    x = pd.ExcelFile("DataSets/Energy Indicators.xls")
    # Load to a dataframe 
    Energy = x.parse(skiprows=17, # Skiping the header information 
                     skipfooter =(38)) # Skipping the 38 latest rows
    
    # Keep only the columns we want
    Energy = Energy[["Unnamed: 2", "Petajoules", "Gigajoules", "%"]]
    
    # Change the columns labels
    Energy.columns = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]
    
    # Convert missing data to np.NaN
    missing_data_columns = ["Energy Supply", "Energy Supply per Capita"]
    Energy[missing_data_columns] = Energy[missing_data_columns].replace("...", np.NaN, regex=False)
    
    # Convert Energy Supply to gigajoules (1 Petajoule = 1,000,000 Gigajoule)
    Energy["Energy Supply"] = Energy["Energy Supply"] * 1000000
    
    # Remove parenthesis and numbers content
    Energy["Country"].replace(" \(.*\)","",regex=True, inplace=True)
    Energy["Country"].replace("([0-9]+)$", "", regex=True, inplace=True)
    
    # Rename a list of countries:
    old_names = ["Republic of Korea",
                 "United States of America",
                 "United Kingdom of Great Britain and Northern Ireland",
                 "China, Hong Kong Special Administrative Region"]
    
    new_names = ["South Korea", 
                 "United States", 
                 "United Kingdom", 
                 "Hong Kong"]
    for n in range(len(old_names)):
        Energy["Country"].replace(old_names[n], new_names[n], inplace=True)
    
    # Load GDP data
    GDP = pd.read_csv("DataSets/world_bank.csv", skiprows=4)
    
    # Rename Country Name column
    GDP.rename(columns={"Country Name": "Country"}, inplace=True)
    
    # Rename a list of countries:
    old_names = ["Korea, Rep.", 
                 "Iran, Islamic Rep.",
                 "Hong Kong SAR, China"]
    new_names = ["South Korea", 
                 "Iran",
                 "Hong Kong"]
    
    for n in range(len(old_names)): 
        GDP["Country"].replace(old_names[n], new_names[n], inplace=True)
        
    # Load the ScimEn data from an excel file
    x = pd.ExcelFile("DataSets/scimagojr-3.xlsx")
    
    # Load to a dataframe
    ScimEn = x.parse()
    
    # Keep only 2006-2015 columns in GDP
    columns_to_keep = ["Country", "2006", "2007", "2008", "2009", "2010", "2011", 
                       "2012", "2013", "2014", "2015"]
    GDP = GDP[columns_to_keep]
    
    # Keep all columns except Region in ScimEn but only the top 15 rank countries
    ScimEn.drop("Region", axis=1, inplace=True)
    ScimEn_Top_15 = ScimEn[:15]
    
    # Merge ScimEn and Energy intersection 
    intersection_1 = pd.merge(ScimEn_Top_15, # left dataframe
                              Energy, # right dataframe
                              how="inner", # Intersection of both
                              left_on="Country", # left index to intersect
                              right_on="Country") # right index to intersect
    
    second_merge = pd.merge(intersection_1, # Left dataframe
                            GDP, # Right dataframe
                            how="inner", # Intersection
                            left_on="Country", # Left index to intersect
                            right_on="Country") # Right index to intersect
    # Change index
    second_merge.set_index("Country", inplace=True)
    
    # Sort by Rank
    second_merge.sort_values(by=["Rank"], inplace=True)
    
    return second_merge
    
    raise NotImplementedError()

In [45]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,273437,272374,2336764,1615239,8.55,245,1.271910e+11,93.0,19.754910,2.752132e+12,3.550343e+12,4.594307e+12,5.101703e+12,6.087164e+12,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13
United States,2,175891,172431,2230544,724472,12.68,363,9.083800e+10,286.0,11.570980,1.381461e+13,1.445186e+13,1.471284e+13,1.444893e+13,1.499205e+13,1.554258e+13,1.619701e+13,1.678485e+13,1.752716e+13,1.823830e+13
India,3,55082,53775,463165,162944,8.41,181,3.319500e+10,26.0,14.969080,9.402599e+11,1.216735e+12,1.198896e+12,1.341887e+12,1.675615e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12
Japan,4,50523,50065,488062,119930,9.66,193,1.898400e+10,149.0,10.232820,4.530377e+12,4.515265e+12,5.037908e+12,5.231383e+12,5.700098e+12,6.157460e+12,6.203213e+12,5.155717e+12,4.850414e+12,4.389476e+12
United Kingdom,5,43389,42284,615670,111290,14.19,226,7.920000e+09,124.0,10.600470,2.706015e+12,3.093407e+12,2.921432e+12,2.412390e+12,2.481580e+12,2.659882e+12,2.704017e+12,2.783251e+12,3.065521e+12,2.932785e+12
Germany,6,38739,38013,433148,95145,11.18,196,1.326100e+10,165.0,17.901530,2.992197e+12,3.421229e+12,3.730028e+12,3.397791e+12,3.396354e+12,3.744409e+12,3.527345e+12,3.732743e+12,3.883920e+12,3.356236e+12
Russian Federation,7,36735,36560,115938,54993,3.16,90,3.070900e+10,214.0,17.288680,9.899305e+11,1.299706e+12,1.660846e+12,1.222644e+12,1.524917e+12,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12
Canada,8,33472,32863,568080,100953,16.97,227,1.043100e+10,296.0,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,27983,26940,352993,87828,12.61,166,6.530000e+09,109.0,33.667230,1.947920e+12,2.210293e+12,2.398857e+12,2.191242e+12,2.134018e+12,2.291991e+12,2.087077e+12,2.141315e+12,2.159134e+12,1.835899e+12


In [305]:

import pandas as pd

# Load the Energy data from an excel file
x = pd.ExcelFile("DataSets/Energy Indicators.xls")
# Load to a dataframe 
Energy = x.parse(skiprows=17, # Skiping the header information 
                 skipfooter =(38)) # Skipping the 38 latest rows

# Keep only the columns we want
Energy = Energy[["Unnamed: 2", "Petajoules", "Gigajoules", "%"]]

# Change the columns labels
Energy.columns = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]

# Convert missing data to np.NaN
missing_data_columns = ["Energy Supply", "Energy Supply per Capita"]
Energy[missing_data_columns] = Energy[missing_data_columns].replace("...", np.NaN, regex=False)

# Convert Energy Supply to gigajoules (1 Petajoule = 1,000,000 Gigajoule)
Energy["Energy Supply"] = Energy["Energy Supply"] * 1000000

# Remove parenthesis and numbers content
Energy["Country"].replace(" \(.*\)","",regex=True, inplace=True)
Energy["Country"].replace("([0-9]+)$", "", regex=True, inplace=True)

# Rename a list of countries:
old_names = ["Republic of Korea",
             "United States of America",
             "United Kingdom of Great Britain and Northern Ireland",
             "China, Hong Kong Special Administrative Region"]

new_names = ["South Korea", 
             "United States", 
             "United Kingdom", 
             "Hong Kong"]
for n in range(len(old_names)):
    Energy["Country"].replace(old_names[n], new_names[n], inplace=True)

# Load GDP data
GDP = pd.read_csv("DataSets/world_bank.csv", skiprows=4)

# Rename Country Name column
GDP.rename(columns={"Country Name": "Country"}, inplace=True)

# Rename a list of countries:
old_names = ["Korea, Rep.", 
             "Iran, Islamic Rep.",
             "Hong Kong SAR, China"]
new_names = ["South Korea", 
             "Iran",
             "Hong Kong"]

for n in range(len(old_names)): 
    GDP["Country"].replace(old_names[n], new_names[n], inplace=True)

# Load the ScimEn data from an excel file
x = pd.ExcelFile("DataSets/scimagojr-3.xlsx")

# Load to a dataframe
ScimEn = x.parse()

# Keep only 2006-2015 columns in GDP
columns_to_keep = ["Country", "2006", "2007", "2008", "2009", "2010", "2011", 
                   "2012", "2013", "2014", "2015"]
GDP = GDP[columns_to_keep]


In [306]:
intersection_1 = pd.merge(ScimEn_Top_15, # left dataframe
                           Energy, # right dataframe
                           how="inner", # Intersection of both
                           left_on="Country", # left index to intersect
                           right_on="Country") # right index to intersect
    
intersection_2 = pd.merge(intersection_1, # Left dataframe
                            GDP, # Right dataframe
                            how="inner", # Intersection
                            left_on="Country", # Left index to intersect
                            right_on="Country") # Right index to intersect

In [314]:
type(len(union_1) - len(intersection_1))

int

In [312]:
def answer_two():
    
    union_1 = pd.merge(ScimEn, # left dataframe
                       Energy, # right dataframe
                       how="outer", # Intersection of both
                       left_on="Country", # left index to union
                       right_on="Country") # right index to union
    
    union_2 = pd.merge(union_1, # left dataframe
                       GDP, # right dataframe
                       how="outer", # Intersection of both
                       left_on="Country", # left index to union
                       right_on="Country") # right index to union
    
    return len(union_1) - len(intersection_1)
    
    raise NotImplementedError()

In [313]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*


In [89]:
    # Recover the DataFrame with the top 15 countries
    top_15_countries = answer_one()
    
    # Keep only the lates 10 columns (2006-2015)
    top_15_countries = top_15_countries.iloc[:,-10:]
    
    # Compute AVG using .apply() function
    top_15_countries["AVG"] = top_15_countries.apply(lambda x: np.mean(x), axis=1)
    
    # Create a list with the countries names
    countries = top_15_countries.index.tolist()
    
    # Create a series with countries and AVG
    avgGDP = pd.Series(top_15_countries["AVG"], index=countries)
    
    # Sort values by AVG
    avgGDP.sort_values(ascending=False, inplace=True)

In [90]:
avgGDP

United States         1.567102e+13
China                 6.927702e+12
Japan                 5.177131e+12
Germany               3.518225e+12
United Kingdom        2.776028e+12
France                2.687371e+12
Italy                 2.139775e+12
Brazil                1.988885e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.398587e+12
South Korea           1.221372e+12
Australia             1.206526e+12
Iran                  4.389286e+11
Name: AVG, dtype: float64

In [66]:
GDP.head(30)

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,AVG
Country,,,,,,,,,,,
World,5.152118e+13,5.805852e+13,6.370874e+13,6.043672e+13,6.616266e+13,7.347986e+13,7.517281e+13,7.733191e+13,7.946886e+13,7.523360e+13,6.805749e+13
High income,3.973002e+13,4.344905e+13,4.618018e+13,4.331528e+13,4.540732e+13,4.900422e+13,4.905922e+13,4.965807e+13,5.075263e+13,4.796578e+13,4.645218e+13
OECD members,3.958351e+13,4.332843e+13,4.592130e+13,4.299680e+13,4.511536e+13,4.851559e+13,4.849080e+13,4.914922e+13,5.019265e+13,4.738321e+13,4.606769e+13
Post-demographic dividend,3.751869e+13,4.093986e+13,4.327912e+13,4.078706e+13,4.255073e+13,4.568593e+13,4.564102e+13,4.610186e+13,4.706094e+13,4.455805e+13,4.341233e+13
IDA & IBRD total,1.230543e+13,1.523620e+13,1.828849e+13,1.776448e+13,2.149306e+13,2.530528e+13,2.692308e+13,2.852324e+13,2.956536e+13,2.801529e+13,2.234199e+13
Europe & Central Asia,1.815163e+13,2.120807e+13,2.328879e+13,2.048889e+13,2.099118e+13,2.326775e+13,2.243208e+13,2.344114e+13,2.376427e+13,2.047613e+13,2.175099e+13
Low & middle income,1.159816e+13,1.437022e+13,1.720855e+13,1.678924e+13,2.036571e+13,2.416828e+13,2.574497e+13,2.731864e+13,2.825061e+13,2.682727e+13,2.126416e+13
Middle income,1.137800e+13,1.410098e+13,1.688263e+13,1.645715e+13,1.998626e+13,2.375489e+13,2.531525e+13,2.685225e+13,2.775079e+13,2.633324e+13,2.088114e+13
IBRD only,1.138467e+13,1.415157e+13,1.699432e+13,1.648609e+13,2.001910e+13,2.364499e+13,2.514105e+13,2.656234e+13,2.747101e+13,2.594090e+13,2.077960e+13


In [91]:
def answer_three():
    
    # Recover the DataFrame with the top 15 countries
    top_15_countries = answer_one()
    
    # Keep only the lates 10 columns (2006-2015)
    top_15_countries = top_15_countries.iloc[:,-10:]
    
    # Compute AVG using .apply() function
    top_15_countries["AVG"] = top_15_countries.apply(lambda x: np.mean(x), axis=1)
    
    # Create a list with the countries names
    countries = top_15_countries.index.tolist()
    
    # Create a series with countries and AVG
    avgGDP = pd.Series(top_15_countries["AVG"], index=countries)
    
    # Sort values by AVG
    avgGDP.sort_values(ascending=False, inplace=True)
    
    return avgGDP
    
    raise NotImplementedError()

In [92]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"

### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*